In [503]:
import dlib
import cv2
import os
import pandas as pd
import numpy as np
import csv

In [504]:
#df = pd.DataFrame()

In [505]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [506]:
def convertToGray(img): # function that convert image to grayscale.
	return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [507]:
def get_landmarks(imag):
	landmarks = []
	detections = detector(imag, 1)
    
	for k,d in enumerate(detections): #For all detected face instances individually
		shape = predictor(imag, d) #Draw Facial Landmarks with the predictor class
		xlist = []
		ylist = []
		for i in range(0,68): #Store X and Y coordinates in two lists
			xlist.append(float(shape.part(i).x))
			ylist.append(float(shape.part(i).y))

		for x, y in zip(xlist, ylist): #Store all landmarks in one list in the format x1,y1,x2,y2,etc.
			landmarks.append(x)
			landmarks.append(y)
            
	if len(detections) > 0:
		return landmarks
	else: #If no faces are detected, return error message to other function to handle
		landmarks = "error"
		return landmarks

In [508]:
def extractFrames(pathIn, pathOut):
	df = pd.DataFrame()
	os.mkdir(pathOut)

	emotion = []
	land_array = []
	col = []
        
	j = 1
	k1 = 1
	k2 = 1
	for j in range(1, 137):
		if (j%2) == 0:
			k1 = j/2
			col.append("land_%d_y" % k1)
		else:
			k2 = (j/2) + 0.5
			col.append("land_%d_x" % k2)
		j += 1
#	print(col)
    
	cap = cv2.VideoCapture(pathIn)
	count = 0
	bias = 50
	while (cap.isOpened()): # o contador vai tirar apenas os frames multiplos de 10 (1 a cada 10 frames)
		# Capture frame-by-frame       
		ret, frame = cap.read()

		if ret == True and (count % 10) == 0:
			print('Read %d frame: ' % count, ret)

			frame = convertToGray(frame) # convert image to gray
			# uses the classifier HaarCascade to extract the faces.
			haar_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
			faces = haar_face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5) # detecta quantas faces ha na imagem

			for (x,y,w,h) in faces: # Para cada face faz um corte
				crop_img = frame[y-bias: y+h+bias, x-bias: x+w+bias]

			landmark = get_landmarks(crop_img) # Aqui recebo lista com landmarks.
			#print(type(landmark)) # Podemos ver com isso que a variavel retornada pela funcao get_landmarks eh do tipo list. 
#			print(len(landmark)) # Tamanho da lista
			land_array = pd.DataFrame([landmark], index = ["frame_%d" % count], columns = col)
			df = df.append(land_array)
            
			clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
			clahe_image = clahe.apply(crop_img) # Aplicação do Clahe a imagem ja em cinza
			detections = detector(clahe_image, 1)#Detect the faces in the image
			for k,d in enumerate(detections): #For each detected face
            			shape = predictor(clahe_image, d) #Get coordinates
            			for i in range(1,68): #There are 68 landmark points on each face
                     			#For each point, draw a red circle with thickness2 on the original frame
                    			cv2.circle(crop_img, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2)

			cv2.imwrite(os.path.join(pathOut, "frame{:d}.jpg".format(count)), crop_img)  # save crop_img as JPEG file

			count += 1
		elif (count % 10) != 0:
			count += 1
		else:
			break

	k = 0
	num_em = extract_label(pathIn)
	while (k < (count/10)):            
		emotion.append(num_em)
		k += 1
	df.insert(136, "label", emotion, True) 

	print(df)
	return df
	# When everything done, release the capture
	cap.release()
	#cv2.destroyAllWindows()


In [509]:
def extract_label(video_name): # Essa função extrai do nome do video o numero da emocao.
	i = 0
	num = 0
	while(video_name[i] != '_'):
		i += 1
	if i == 3:
		num = ord(video_name[2]) - 48
	if i == 4:
		x = ord(video_name[3]) - 48 # pego o digito da unidade 
		y = (ord(video_name[2]) - 48) * 10 # pego o digito dos decimais
		num = x + y
	return num

In [510]:
def create_csv(j):
	export_csv = df.to_csv('dataframe_video{:d}.csv'.format(j)) 
#	f = open("guru99.txt","w+")
#	

In [511]:
def main():
	i = 1
	while(i < 3):
#	dataFrame = extractFrames('Em1_Fala1_CarolinaHolly.mp4', 'A16')
		dataFrame = extractFrames('Em{:d}_.mp4'.format(i), 'frames_em{:d}'.format(i))
		create_csv(i)
		i += 1
	file = pd.read_csv("dataframe_video1.csv", header=None)
	file.head()

In [512]:
if __name__=="__main__":
	main()

Read 0 frame:  True
Read 10 frame:  True
Read 20 frame:  True
Read 30 frame:  True
Read 40 frame:  True
Read 50 frame:  True
Read 60 frame:  True
Read 70 frame:  True
Read 80 frame:  True
Read 90 frame:  True
Read 100 frame:  True
Read 110 frame:  True
Read 120 frame:  True
Read 130 frame:  True
Read 140 frame:  True
Read 150 frame:  True
Read 160 frame:  True
Read 170 frame:  True
Read 180 frame:  True
Read 190 frame:  True
Read 200 frame:  True
Read 210 frame:  True
Read 220 frame:  True
Read 230 frame:  True
Read 240 frame:  True
Read 250 frame:  True
Read 260 frame:  True
Read 270 frame:  True
Read 280 frame:  True
Read 290 frame:  True
Read 300 frame:  True
Read 310 frame:  True
Read 320 frame:  True
Read 330 frame:  True
Read 340 frame:  True
Read 350 frame:  True
Read 360 frame:  True
Read 370 frame:  True
Read 380 frame:  True
Read 390 frame:  True
           land_1_x  land_1_y  land_2_x  land_2_y  land_3_x  land_3_y  \
frame_0        84.0     213.0      88.0     263.0      96.

Read 10 frame:  True
Read 20 frame:  True
Read 30 frame:  True
Read 40 frame:  True
Read 50 frame:  True
Read 60 frame:  True
Read 70 frame:  True
Read 80 frame:  True
Read 90 frame:  True
Read 100 frame:  True
Read 110 frame:  True
Read 120 frame:  True
Read 130 frame:  True
Read 140 frame:  True
Read 150 frame:  True
Read 160 frame:  True
Read 170 frame:  True
Read 180 frame:  True
Read 190 frame:  True
Read 200 frame:  True
Read 210 frame:  True
Read 220 frame:  True
Read 230 frame:  True
Read 240 frame:  True
Read 250 frame:  True
Read 260 frame:  True
Read 270 frame:  True
Read 280 frame:  True
Read 290 frame:  True
Read 300 frame:  True
Read 310 frame:  True
Read 320 frame:  True
Read 330 frame:  True
Read 340 frame:  True
Read 350 frame:  True
Read 360 frame:  True
Read 370 frame:  True
Read 380 frame:  True
Read 390 frame:  True
Read 400 frame:  True
Read 410 frame:  True
Read 420 frame:  True
Read 430 frame:  True
Read 440 frame:  True
           land_1_x  land_1_y  land_2_x  